# Calculate based ssGSEA methods TMEcellularity scores

Script related to figure 3b

### Aim:
- Calculate purity scores for Bindea, Davoli, Danaher, and ConsensusTME

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Load packages

In [1]:
library(GSVA)
library(BBmisc)
library(nortest)

Warning message:
“package ‘GSVA’ was built under R version 3.5.1”
Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/3/")

## Load expression data

In [ ]:
mix <- "../../Data/1/TCGAov_RNAseq.txt"

In [3]:
data <- read.csv(mix,
                 sep='\t',
                 header=T,
                 row.names='GeneSymbol')

In [4]:
head(data)

,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,TCGA.09.0367.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
A1BG,66.4695,65.5664,41.6412,187.0368,23.9295,32.8123,46.3418,162.8522,88.5518,192.7354,⋯,63.7012,99.1485,38.3684,140.3668,94.7103,70.4394,43.1736,33.6854,27.9604,16.2092
A1BG-AS1,36.3243,28.1315,23.2465,114.6008,10.4957,20.1900,17.2007,83.0906,65.6758,55.1265,⋯,30.3602,71.1797,26.0239,77.2415,61.4890,106.1623,47.8864,25.3482,17.9853,42.9450
A1CF,0.0000,0.0000,0.3310,0.0000,0.0000,0.0000,0.0000,0.9034,0.0000,0.0000,⋯,0.0000,0.0000,0.0000,1.9057,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A2M,5899.8279,9384.4401,3350.4207,1455.2316,3999.3792,3224.5797,1986.1593,1487.3247,10557.8325,4603.9013,⋯,10264.1373,7220.7664,7200.1144,1819.9428,6179.7358,5365.7972,8773.5936,10006.7877,1888.1746,4205.6055
A2M-AS1,118.4566,111.0026,71.3613,67.8607,52.1501,224.7582,182.0650,120.2838,53.6419,92.9112,⋯,146.6130,45.8648,64.4638,28.5565,100.6605,23.5625,84.2153,582.3492,59.8084,143.4910
A2ML1,7.5289,54.6875,5.6263,5.0883,3.3148,3.7421,6.2305,278.2486,4.7763,653.6058,⋯,1.7993,4.6124,0.4768,17.1510,15.6109,2.4461,98.3162,269.5768,4.5092,224.5290


In [5]:
data_mtx <- as.matrix(data)

## Load Bindea TME cellularity gene set

In [6]:
Bindea_TMEcell <- read.csv('../../Data/3/BindeaTMEcellularity_GeneSet.txt',
                           sep='\t',
                           header=F,
                           row.names=1)

In [7]:
head(Bindea_TMEcell)

,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,⋯,V541,V542,V543,V544,V545,V546,V547,V548,V549,V550
Bindea_TMEcell,A2M,ABCB4,ABCC4,ABCG2,ABHD2,ABT1,ACACB,ACVRL1,ADARB1,ADCY1,⋯,ZEB1,ZFP36L2,ZNF205,ZNF22,ZNF423,ZNF528,ZNF609,ZNF747,ZNF764,ZNF91


In [8]:
Bindea_TMEcell <- convertRowsToList(Bindea_TMEcell)

## Load Davoli TME cellularity gene set

In [9]:
Davoli_TMEcell <- read.csv('../../Data/3/DavoliTMEcellularity_GeneSet.txt',
                           sep='\t',
                           header=F,
                           row.names=1)

In [10]:
head(Davoli_TMEcell)

,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,⋯,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76
Davoli_TMEcell,AMIGO2,ANPEP,BANK1,BCL11B,C1QA,C1QB,C1QC,CCL5,CCL8,CCR5,⋯,RCAN3,RENBP,SAMD3,SERPINB9,SLAMF1,SLC35D1,STYK1,TBX21,TNIP3,TRAT1


In [11]:
Davoli_TMEcell <- convertRowsToList(Davoli_TMEcell)

## Load Danaher purity gene set

In [12]:
Danaher_TMEcell <- read.csv('../../Data/3/DanaherTMEcellularity_GeneSet.txt',
                            sep='\t',
                            header=F,
                            row.names=1)

In [13]:
head(Danaher_TMEcell)

,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,⋯,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61
Danaher_TMEcell,BLK,CCL13,CD163,CD19,CD209,CD244,CD3D,CD3E,CD3G,CD6,⋯,SIGLEC5,SPIB,TBX21,TCL1A,TNFRSF17,TPSAB1,TPSB2,TRAT1,XCL1,XCL2


In [14]:
Danaher_TMEcell <- convertRowsToList(Danaher_TMEcell)

## Load ConsensusTME purity gene set

In [15]:
ConsensusTMEcell <- read.csv('../../Data/3/ConsensusTMEcellularity_GeneSet.txt',
                             sep='\t',
                             header=F,
                             row.names=1)

In [16]:
head(ConsensusTMEcell)

,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,⋯,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200
ConsensusTME_TMEcellularity,ABCA6,ABHD2,ACTR2,AIF1,AIM2,AKAP13,APOBEC3G,ARHGAP15,ASPN,ATG7,⋯,TRAF3IP3,TRAT1,TREM1,TREM2,VASP,VNN2,WAS,WISP1,WNT2,ZAP70


In [17]:
ConsensusTMEcell <- convertRowsToList(ConsensusTMEcell)

## Run ssGSEA

#### Bindea

In [18]:
bin_NES <- gsva(data_mtx,
                Bindea_TMEcell,
                method='ssgsea',
                min.sz=0,
                max.sz=1100,
                ssgsea.norm=T)

Warning message in .local(expr, gset.idx.list, ...):
“338 genes with constant expression values throuhgout the samples.”

Estimating ssGSEA scores for 1 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [19]:
bin_NES <- data.frame("Term"=rownames(bin_NES),
                      bin_NES)

rownames(bin_NES) <- NULL

In [20]:
head(bin_NES)

Term,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
Bindea_TMEcell,0.7232356,1.022811,0.5347471,0.3110132,0.7186156,0.4524902,0.4034813,0.3175915,0.6883937,⋯,0.7706213,0.6830099,0.5379009,0.5738575,0.6737607,0.8239319,0.3792633,0.3534607,0.433556,0.3293046


In [21]:
write.table(bin_NES,
            file='../../Data/3/BindeaTMEcellularity_TCGAov_NES.txt',
            sep='\t',
            row.names=F,
            col.names=T)

#### Davoli

In [22]:
dav_NES <- gsva(data_mtx,
                Davoli_TMEcell,
                method='ssgsea',
                min.sz=0,
                max.sz=1100,
                ssgsea.norm=T)

Warning message in .local(expr, gset.idx.list, ...):
“338 genes with constant expression values throuhgout the samples.”

Estimating ssGSEA scores for 1 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [23]:
dav_NES <- data.frame("Term"=rownames(dav_NES),
                      dav_NES)

rownames(dav_NES) <- NULL

In [24]:
head(dav_NES)

Term,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
Davoli_TMEcell,0.1532845,0.3756873,-0.2540212,-0.5310531,0.07847188,-0.4942905,-0.3853437,-0.5954481,-0.02651432,⋯,-0.2525507,0.04738229,-0.1776927,-0.3375489,-0.0941406,0.05417709,-0.4878406,-0.5095475,-0.2837717,-0.4875616


In [25]:
write.table(dav_NES,
            file='../../Data/3/DavoliTMEcellularity_TCGAov_NES.txt',
            sep='\t',
            row.names=F,
            col.names=T)

#### Danaher

In [26]:
dan_NES <- gsva(data_mtx,
                Danaher_TMEcell,
                method='ssgsea',
                min.sz=0,
                max.sz=1100,
                ssgsea.norm=T)

Warning message in .local(expr, gset.idx.list, ...):
“338 genes with constant expression values throuhgout the samples.”

Estimating ssGSEA scores for 1 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [27]:
dan_NES <- data.frame("Term"=rownames(dan_NES),
                      dan_NES)

rownames(dan_NES) <- NULL

In [28]:
head(dan_NES)

Term,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
Danaher_TMEcell,-0.2815311,-0.0626237,-0.6841806,-0.8771487,-0.3380261,-0.901362,-0.7510459,-0.978688,-0.4207622,⋯,-0.6622266,-0.4253173,-0.5426976,-0.6055369,-0.4281354,-0.3967194,-0.8627661,-0.8463669,-0.7679065,-0.9588643


In [29]:
write.table(dan_NES,
            file='../../Data/3/DanaherTMEcellularity_TCGAov_NES.txt',
            sep='\t',
            row.names=F,
            col.names=T)

#### ConsensusTME

In [30]:
con_NES <- gsva(data_mtx,
                ConsensusTMEcell,
                method='ssgsea',
                min.sz=0,
                max.sz=1100,
                ssgsea.norm=T)

Warning message in .local(expr, gset.idx.list, ...):
“338 genes with constant expression values throuhgout the samples.”

Estimating ssGSEA scores for 1 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [31]:
con_NES <- data.frame("Term"=rownames(con_NES),
                      con_NES)

rownames(con_NES) <- NULL

In [32]:
head(con_NES)

Term,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
ConsensusTME_TMEcellularity,0.3012972,0.5125812,-0.01010583,-0.3239443,0.2317494,-0.3067751,-0.2038634,-0.3396519,0.3721918,⋯,0.09065798,0.2304181,0.05127847,-0.07465664,0.09162138,0.2482765,-0.241627,-0.2770736,-0.1382935,-0.2683264


In [33]:
write.table(con_NES,
            file='../../Data/3/ConsensusTMEcellularity_TCGAov_NES.txt',
            sep='\t',
            row.names=F,
            col.names=T)

# End script